# 作業

實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [1]:
import cv2
import numpy as np

img_path = 'lena.png'

# 以彩色圖片的方式載入
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

## 改變飽和度

- 轉換成 HSV color space, 改變 s channel 的值

In [2]:
# 為了要改變飽和度，我們先把 color space 轉成 HSL 格式 (OpenCV 表示順序是 HLS)
img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
change_percentage = 0.2

# 針對飽和度的值做改變，超過界線 0~1 的都會 bound
# 在 HLS color space 減少飽和度
img_hls_down = img_hls.astype('float32')
img_hls_down[..., -1] = img_hls_down[..., -1]/255 - change_percentage
img_hls_down[img_hls_down[..., -1] < 0] = 0
img_hls_down[..., -1] = img_hls_down[..., -1]*255
img_hls_down = img_hls_down.astype('uint8')

# 在 HLS color space 增加飽和度
img_hls_up = img_hls.astype('float32')
img_hls_up[..., -1] = img_hls_up[..., -1]/255 + change_percentage
img_hls_up[img_hls_up[..., -1] > 1] = 1
img_hls_up[..., -1] = img_hls_up[..., -1]*255
img_hls_up = img_hls_up.astype('uint8')

# 轉回BGR
img_hls_down = cv2.cvtColor(img_hls_down, cv2.COLOR_HLS2BGR)
img_hls_up = cv2.cvtColor(img_hls_up, cv2.COLOR_HLS2BGR)

# 組合圖片 + 顯示圖片
img_hls_change = np.hstack((img, img_hls_down, img_hls_up))
while True:
    cv2.imshow('change saturation', img_hls_change)
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        
        break

: 

## 直方圖均衡

1. case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
2. case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# case 1
# 每個 channel 個別做直方圖均衡
image = cv2.imread("lena.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_r, image_g, image_b = cv2.split(image_rgb)
image_r_equalized = cv2.equalizeHist(image_r)
image_g_equalized = cv2.equalizeHist(image_g)
image_b_equalized = cv2.equalizeHist(image_b)
image_rgb_equalized = cv2.merge((image_r_equalized, image_g_equalized, image_b_equalized))
# 組合經過直方圖均衡的每個 channel

# case 2 - 轉換 color space 後只對其中一個 channel 做直方圖均衡
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
image_hsv[:,:,2] = cv2.equalizeHist(image_hsv[:,:,2])
image_hsv_equalized = cv2.cvtColor(image_hsv, cv2.COLOR_HSV2RGB)

# 組合圖片 + 顯示圖片
image_bgr_equalHist = np.hstack((image_rgb, image_rgb_equalized, image_hsv_equalized))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow('bgr equal histogram', image_bgr_equalHist)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

## 調整對比 / 明亮

In [1]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("lena.png")

add_contrast = cv2.convertScaleAbs(img, alpha=2.0, beta=0)
add_lighness = cv2.convertScaleAbs(img, alpha=1.0, beta=50)

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((img, add_contrast, add_lighness))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow('adjust contrast and brighness', img_contrast_light)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

: 